# Basic Tensorflow

This notebook will familiarize you with the **basic concepts**
of Tensorflow. Each of these concepts could be extended into
its own notebook(s) but because we want to do some actual
machine learning later on, we only briefly touch on each of
the concepts.

Table of Contents:

- [ 1 The Graph](#1-The-Graph)
- [ 2 The Session](#2-The-Session)
- [ 3 The Shapes](#3-The-Shapes)
- [ 4 Variables – bonus!](#4-Variables-%E2%80%93-bonus!)

In [1]:
import tensorflow as tf
# Always make sure you are using running the expected version.
# There are considerable differences between versions...
# We tested this with version 1.4.X
tf.__version__

'1.4.1'

# 1 The Graph

Most important concept with Tensorflow : There is a Graph to which
tensors are attached. This graph is never specified explicitly but
has important consequences for the tensors that are attached to it
(e.g. you cannot connect two tensors that are in different graphs).

The python variable "tensor" is simply a reference to the actual
tensor in the Graph. More precisely, it is a reference to an operation
that will produce a tensor (in the Tensorflow Graph, the nodes are
actually operations and the tensors "flow" on the edges between
the nodes...)

Important note : There is a new simplification of the execution theme
presented in this notebook :
[Tensorflow Eager](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/notebooks/1_basics.ipynb)
-- But since this feature is currently in alpha state and most code
still uses the graph/session paradigm, we won't use the new execution
style (anyways, if you can use the old style the new simplified style
should be a welcome and straight forward simplification...)

In [2]:
# There is always a "graph" even if you haven't defined one.
tf.get_default_graph()

In [3]:
# Store the default graph in a variable for exploration.
graph = tf.get_default_graph()

In [4]:
# Ok let's try to get all "operations" that are currently defined in this
# default graph.
# Remember : Placing the caret at the end of the line and typing <tab> will
# show an auto-completed list of methods...
graph.get_operations() #graph.get_

[]

In [5]:
# Let's create a separate graph:
graph2 = tf.Graph()

# Try to predict what these statements will output.
print tf.get_default_graph() == graph
print tf.get_default_graph() == graph2
with graph2.as_default():
    print tf.get_default_graph() == graph
    print tf.get_default_graph() == graph2

True
False
False
True


In [6]:
# We define our first TENSOR. Fill in your favourite numbers
# You can find documentation to this function here:
# https://www.tensorflow.org/versions/master/api_docs/python/tf/constant

# Try to change data type and shape of the tensor...

favorite_numbers = tf.constant([13, 22, 83])

print favorite_numbers

# (Note that this only prints the "properties" of the tensor
# and not its actual value -- more about this strange behavior
# in the section "The Session".)

Tensor("Const:0", shape=(3,), dtype=int32)


In [7]:
# Remember that graph that is always in the background? All the
# tensors that you defined above have been duefully attached to the
# graph by Tensorflow -- check this out:
# (Also note how the operations are named by default)

graph.get_operations()  # Show graph operations.

[<tf.Operation 'Const' type=Const>]

In [8]:
# Note that above are the OPERATIONS that are the nodes in the
# graph (in our the case the "Const" operation creates a constant
# tensor). The tensors themselves are the EDGES between the nodes,
# and their name is usually the operation's name + ":0".
favorite_numbers.name

u'Const:0'

In [9]:
# Let's say we want to clean up our experimental mess...
# Search on Tensorflow homepage for a command to "reset" the graph:
# https://www.tensorflow.org/api_docs/

### YOUR ACTION REQUIRED:
# Find the right Tensorflow command to reset the graph.
tf.reset_default_graph() #tf.
tf.get_default_graph().get_operations()

[]

In [10]:
# Important note: "resetting" didn't clear our original graph but
# rather replace it with a new graph:
tf.get_default_graph() == graph

False

In [11]:
# Because we cannot define operations across graphs, we need to
# redefine our favorite numbers in the context of the new
# graph:

favorite_numbers = tf.constant([13, 22, 83])

In [12]:
# Now let's do some computations. Actually we don't really execute
# any computation yet (see next section "The Session" for that), but
# rather define how we intend to do computation later on...

# We first multiply our favorite numbers with our favorite multiplier:
favorite_multiplier = tf.constant(7)
# Do you have an idea how to write below multiplication more succinctly?
# Try it! (Hint: operator overloading)
favorite_products = tf.multiply(favorite_multiplier, favorite_numbers)
print 'favorite_products.shape=', favorite_products.shape

# Now we want to add up all the favorite products to a single scalar
# (0-dim tensor).
# There is a Tensorflow function for this. It starts with "reduce"...
# (Use <tab> auto-completion and/or tensorflow documentation)
### YOUR ACTION REQUIRED:
# Find the correct Tensorflow command to sum up the numbers.
favorite_sum = tf.reduce_sum(favorite_products) #favorite_sum = tf.
print 'favorite_sum.shape=', favorite_sum.shape

favorite_products.shape= (3,)
favorite_sum.shape= ()


In [13]:
# Because we really like our "first" favorite number we add this number
# again to the sum:

favorite_sum_enhanced = favorite_sum + favorite_numbers[0]
# See how we used Python's overloaded "+" and "[]" operators?

# You could also define the same computation using Tensorflow
# functions only:
# favorite_sum_enhanced = tf.add(favorite_sum, tf.slice(favorite_numbers, [0], [1]))

In [14]:
# Of course, it's good practice to avoid a global invisible graph, and
# you can use a Python "with" block to explicitly specify the graph for
# a codeblock:
with tf.Graph().as_default():
    within_with = tf.constant([1, 2, 3], name='within_with')
    print 'within with:'
    print tf.get_default_graph()
    print within_with
    print tf.get_default_graph().get_operations()
print '\noutside with:'
print tf.get_default_graph()
print within_with
print tf.get_default_graph().get_operations()

# You can execute this cell multiple times without messing up any graph.
# Note that you won't be able to connect the tensor to other tensors
# because we didn't store a reference to the graph of the with statement.

within with:
Tensor("within_with:0", shape=(3,), dtype=int32)
[<tf.Operation 'within_with' type=Const>]

outside with:
Tensor("within_with:0", shape=(3,), dtype=int32)
[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'Const_2' type=Const>, <tf.Operation 'Sum' type=Sum>, <tf.Operation 'strided_slice/stack' type=Const>, <tf.Operation 'strided_slice/stack_1' type=Const>, <tf.Operation 'strided_slice/stack_2' type=Const>, <tf.Operation 'strided_slice' type=StridedSlice>, <tf.Operation 'add' type=Add>]


In [15]:
%%writefile _derived/2_visualize_graph.py
# (Written into separate file for sharing between notebooks.)

# Let's visualize our graph!
# Tip: to make your graph more readable you can add a
# name="..." parameter to the individual Ops.

# src: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb

import numpy as np
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

Overwriting _derived/2_visualize_graph.py


In [16]:
# (Load code from previous cell -- make sure to have executed above cell first.)
%run -i _derived/2_visualize_graph.py

show_graph(tf.get_default_graph())

# 2 The Session

So far we have only setup our computational Graph -- If you want to actually
*do* any computations, you need to attach the graph to a Session.

In [17]:
# The only difference to a "normal" session is that the interactive
# session registers itself as default so .eval() and .run() methods
# know which session to use...
interactive_session = tf.InteractiveSession()

In [18]:
# Hooray -- try printing other tensors of above to see the intermediate
# steps. What is their type and shape ?
print favorite_sum.eval()

826


In [19]:
# Note that the session is also connected to a Graph, and if no Graph
# is specified then it will connect to the default Graph. Try to fix
# the following code snippet:

graph2 = tf.Graph()
with graph2.as_default():
    graph2_tensor = tf.constant([1])
with tf.Session(graph=graph2) as sess: #with tf.Session() as sess:
    print graph2_tensor.eval()

[1]


In [20]:
# Providing input to the graph: The value of any tensor can be overwritten
# by the "feed_dict" parameter provided to Session's run() method:
a = tf.constant(1)
b = tf.constant(2)
a_plus_b = tf.add(a, b)
print interactive_session.run(a_plus_b)
print interactive_session.run(a_plus_b, feed_dict={a: 123000, b:456})

3
123456


In [21]:
# It's good practice not to override just any tensor in the graph, but to
# rather use "tf.placeholder" that indicates that this tensor must be
# provided through the feed_dict:
placeholder = tf.placeholder(tf.int32)
placeholder_double = 2 * placeholder
### YOUR ACTION REQUIRED:
# Modify below command to make it work.
print placeholder_double.eval(feed_dict={placeholder:21}) #print placeholder_double.eval()

42


# 3 The Shapes

Another basic skill with Tensorflow is the handling of shapes. This
sounds pretty simple but you will be surprised by how much time of
your Tensorflow coding you will spend on massaging Tensors in the
right form...

Here we go with a couple of exercises with increasing difficulty...

Please refer to the Tensorflow documentation
[Tensor Transformations](https://www.tensorflow.org/versions/master/api_guides/python/array_ops#Shapes_and_Shaping)
for useful functions.

In [22]:
tensor12 = tf.constant([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
print tensor12
batch = tf.placeholder(tf.int32, shape=[None, 3])
print batch

Tensor("Const_5:0", shape=(12,), dtype=int32)
Tensor("Placeholder_1:0", shape=(?, 3), dtype=int32)


In [23]:
# Tensor must be of same datatype. Try to change the datatype
# of one of the tensors to fix the ValueError...
multiplier = tf.constant(1.5)
### YOUR ACTION REQUIRED:
# Fix error below.
tf.cast(tensor12, tf.float32) * multiplier #tensor12 * multiplier

<tf.Tensor 'mul_1:0' shape=(12,) dtype=float32>

In [24]:
# What does tf.squeeze() do? Try it out on tensor12_3!
tensor12_3 = tf.reshape(tensor12, [3, 2, 2, 1])
### YOUR ACTION REQUIRED:
# Checkout the effects of tf.squeeze()
print tf.squeeze(tensor12_3).shape, tensor12_3.shape #print tensor12_3.shape

(3, 2, 2) (3, 2, 2, 1)


In [25]:
# This cell is about accessing individual elements of a 2D tensor:
batch = tf.constant([[1, 2, 3, 0, 0],
                     [2, 4, 6, 8, 0],
                     [3, 6, 0, 0, 0]])
# Note that individual elements have lengths < batch.shape[1] but
# are zero padded.
lengths = tf.constant([3, 4, 2])

# The FIRST elements can be accessed by using Python's
# overloaded bracket indexing OR the related tf.slice():
print 'first elements:'
print batch[:, 0].eval()
print tf.slice(batch, [0, 0], [3, 1]).eval()

first elements:
[1 2 3]
[[1]
 [2]
 [3]]


In [26]:
# Accessing the LAST (non-padded) element within every sequence is
# somewhat more involved -- you need to specify both the indices in
# the first and the second dimension and then use tf.gather_nd():
### YOUR ACTION REQUIRED:
# Define provide the correct expression for indices_0 and indices_1.
indices_0 = range(3) #indices_0 =
indices_1 = lengths - 1 #indices_1 =
print 'last elements:'
print tf.gather_nd(batch, tf.transpose([indices_0, indices_1])).eval()

last elements:
[3 8 6]


In [27]:
# Below you have an integer tensor and then an expression that is set True
# for all elements that are odd. Try to print those elements using the
# operations tf.where() and tf.gather()
numbers = tf.range(1, 11)
odd_condition = tf.logical_not(tf.equal(0, tf.mod(numbers, 2)))
### YOUR ACTION REQUIRED:
# Define provide the correct expression for odd_indices and odd_numbers.
odd_indices = tf.where(odd_condition) #odd_indices =
odd_numbers = tf.gather(numbers, odd_indices) #odd_numbers =
print odd_numbers.eval()

[[1]
 [3]
 [5]
 [7]
 [9]]


In [28]:
# "Dynamic shapes" : This feature is mainly used for variable size batches.
# "Dynamic" means that one (or multiple) dimensions are not specified
# before graph execution time (when running the graph with a session).
batch_of_pairs = tf.placeholder(dtype=tf.int32, shape=(None, 2))

# Note how the "unknown" dimension displays as a "?".
print batch_of_pairs

Tensor("Placeholder_2:0", shape=(?, 2), dtype=int32)


In [29]:
# So we want to reshape the batch of pairs into a batch of quadruples.
# Since we don't know the batch size at runtime we will use the special
# value "-1" (meaning "as many as needed") for the first dimension.
# (Note that this wouldn't work for batch_of_triplets.)
### YOUR ACTION REQUIRED:
# Complete next line.
batch_of_quadruples = tf.reshape(batch_of_pairs, [-1, 4]) #batch_of_quadruples = tf.reshape(batch_of_pairs, 

# Test run our batch of quadruples:
print batch_of_quadruples.eval(feed_dict={
    batch_of_pairs: [[1,2], [3,4], [5,6], [7,8]]})

[[1 2 3 4]
 [5 6 7 8]]


In [30]:
# Dynamic shapes cannot be accessed at graph construction time;
# accessing the ".shape" attribute (which is equivalent to the
# .get_shape() method) will return a "TensorShape" with "Dimension(None)".
batch_of_pairs.shape

# i.e. .shape is a property of every tensor that can contain
# values that are not specified -- Dimension(None)

TensorShape([Dimension(None), Dimension(2)])

In [31]:
# i.e. first dimension is dynamic and only known at runtime
batch_of_pairs.shape[0].value == None

True

In [32]:
# The actual dimensions can only be determined at runtime
# by calling tf.shape() -- the output of the tf.shape() Op
# is a tensor like any other tensor whose value is only known
# at runtime (when also all dynamic shapes are known).
batch_of_pairs_shape = tf.shape(batch_of_pairs)
batch_of_pairs_shape.eval(feed_dict={
    batch_of_pairs: [[1, 2]]
})

# i.e. tf.shape() is an Op that takes a tensor (that might have
# a dynamic shape or not) as input and outputs another tensor
# that fully specifies the shape of the input tensor.

array([1, 2], dtype=int32)

In [33]:
# So you think shapes are easy, right?
# Well... Then here we go with a real-world shape challenge!
#
# (You probably won't have time to finish this challenge during
# the workshop; come back to this later and don't feel bad about
# consulting the solution...)
#
# Imagine you have a recurrent neural network that outputs a "sequence"
# tensor with dimension [?, max_len, ?], where
# - the first (dynamic) dimension is the number of elements in the batch
# - the second dimension is the maximum sequence length
# - the third (dynamic) dimension is the number of number per element
#
# The actual length of every sequence in the batch (<= max_len) is also
# specified in the tensor "lens" (length=number of elements in batch).
#
# The task at hand is to extract the "nth" element of every sequence.
# The resulting tensor "last_elements" should have the shape [?, ?],
# matching the first and third dimension of tensor "sequence".
#
# Hint: The idea is to reshape the "sequence" to "partially_flattened"
# and then construct a "idxs" tensor (within this partially flattened
# tensor) that returns the requested elements.
#
# Handy functions:
# tf.gather()
# tf.range()
# tf.reshape()
# tf.shape()

lens = tf.placeholder(dtype=tf.int32, shape=(None,))
max_len = 5
sequences = tf.placeholder(dtype=tf.int32, shape=(None, max_len, None))
### YOUR ACTION REQUIRED:
# Find the correct expression for below tensors.
batch_size = tf.shape(sequences)[0] #batch_size = 
hidden_state_size = tf.shape(sequences)[2] #hidden_state_size = 
idxs = tf.range(0, batch_size) * max_len + (lens - 1) #idxs = 
partially_flattened = tf.reshape(sequences, [-1, hidden_state_size]) #partially_flattened =
last_elements = tf.gather(partially_flattened, idxs) #last_elements =

sequences_data = [
    [[1,1], [1,1], [2,2], [0,0], [0,0]],
    [[1,1], [1,1], [1,1], [3,3], [0,0]],
    [[1,1], [1,1], [1,1], [1,1], [4,4]],
]
lens_data = [3, 4, 5]
# Should output [[2,2], [3,3], [4,4]]
last_elements.eval(feed_dict={sequences: sequences_data, lens: lens_data})

array([[2, 2],
       [3, 3],
       [4, 4]], dtype=int32)

# 4 Variables – bonus!

So far all our computations have been purely stateless. Obviously,
programming become much more fun once we add some state to our code...
Tensorflow's **variables** encode state that persists between calls to
`Session.run()`.

The confusion with Tensorflow and variables comes from the fact that we
usually "execute" the graph from within Python by running some nodes of
the graph -- via `Session.run()` -- and that variable assignments are also
encoded through nodes in the graph that only get executed if we ask the
value of one of its descendants (see explanatory code below).

Tensorflow's overview of
[variable related functions](https://www.tensorflow.org/versions/r1.0/api_guides/python/state_ops#Variables),
the
[variable HOWTO](https://www.tensorflow.org/versions/r1.0/programmers_guide/variables),
and the
[variable guide](https://www.tensorflow.org/programmers_guide/variables).

And finally some notes on [sharing variables](https://www.tensorflow.org/api_guides/python/state_ops#Sharing_Variables).

In [34]:
counter = tf.Variable(0)
increment_counter = tf.assign_add(counter, 1)
with tf.Session() as sess:
    # Something is missing here...
    # -> Search the world wide web for the error message...
    ### YOUR ACTION REQUIRED:
    # Add a statement that fixes the error.
    sess.run([tf.global_variables_initializer()]) #
    print increment_counter.eval()
    print increment_counter.eval()
    print increment_counter.eval()

1
2
3


In [35]:
# Same conditions apply when we use our global interactive session...
interactive_session.run([tf.global_variables_initializer()])
print increment_counter.eval()

1


In [36]:
# Execute this cell multiple times and note how our global interactive
# sessions keeps state between cell executions.
print increment_counter.eval()

2


In [37]:
# Usually you would create variables with tf.get_variable() which makes
# it possible to "look up" variables later on.

# For a change let's not try to fix a code snippet but rather to make it
# fail:
# 1. What happens if the block is not wrapped in a tf.Graph()?
# 2. What happens if reuse= is not set?
# 3. What happens if dtype= is not set?

with tf.Graph().as_default():
    with tf.variable_scope('counters'):
        counter1 = tf.get_variable('counter1', initializer=1)
        counter2 = tf.get_variable('counter2', initializer=2)
        counter3 = tf.get_variable('counter3', initializer=3)
    with tf.Session() as sess:
        sess.run([tf.global_variables_initializer()])
        print counter1.eval()
        with tf.variable_scope('counters', reuse=True):
            print tf.get_variable('counter2', dtype=tf.int32).eval()

1
2
